In [1]:
import os
import sys
sys.path.append("../")
%load_ext autoreload

In [2]:
os.listdir("..\data")
path = "..\data"
targets = []
for folder in os.listdir(path):
    for f in os.listdir(os.path.join(path, folder)):
        #if f.endswith(".png") or f.endswith(".jpg"):
        if f.endswith(".pdf"):
            targets.append(os.path.join(path, folder, f))
print(targets)

['..\\data\\pdfs\\005-366-fey-squirrel-and-tomb-guardian-compressed.pdf', '..\\data\\pdfs\\006-366-flying-spider-compressed.pdf', '..\\data\\pdfs\\007-366-giant-wolf-compressed.pdf', '..\\data\\pdfs\\008-366-quogwat-compressed.pdf', '..\\data\\pdfs\\009-366-arnitiko-compressed.pdf', '..\\data\\pdfs\\010-366-waterspout-compressed.pdf', '..\\data\\pdfs\\011-366-gargantuan-hamster_and_squid-compressed.pdf', '..\\data\\pdfs\\012-366-el-cadejo-compressed.pdf', '..\\data\\pdfs\\922210-OUTCLASSED_Expansion__Tashas_Panopticon_of_Nemeses.pdf', '..\\data\\pdfs\\Episode 1 - In Scarlet Flames.pdf', '..\\data\\pdfs\\Warlock 15 - Boss Monsters.pdf']


In [12]:
%autoreload 2

from data_loaders.textract_image_loader import TextractImageLoader
from data_loaders.pdf_loader import PDFLoader

from outputs.default_writer import DefaultWriter
from outputs.pluto_writer import PlutoWriter

from extractor import StatblockExtractor
from utils.config import get_config, get_argparser
from utils.logger import get_logger

from fifthedition.creature_printer import pretty_format_creature

# Get arguments
parser = get_argparser()
args = parser.parse_args(["..\data\clockwork_drake\clockwork_drake.jpg", "--config", r"..\default.conf", "--cache",  r"..\.cache"])
print(args)

# Get config file
config = get_config(args)
config.set("default", "use_cache", "true")

logger = get_logger(args.debug, args.notebook, args.logs)

se = StatblockExtractor(config, logger)
se.register_data_loader(TextractImageLoader)
se.register_data_loader(PDFLoader)

se.register_output_writer(DefaultWriter, append=True)
se.register_output_writer(PlutoWriter, append=True)

Namespace(targets=['..\\data\\clockwork_drake\\clockwork_drake.jpg'], source=None, authors=None, url=None, output=None, output_format='default', cache='..\\.cache', config='..\\default.conf', logs=None, debug=False, notebook=False)


In [15]:
args.output = "../outclassed.json"
args.output_format = "5et"

se.select_writer(args.output_format)

for target in targets[-3:-2]:
    print(target)
    logger.info("Loading creatures from {}".format(target))
    
    results = se.parse(target, args.output, pages=[4,5,6,7,8,9,10])
    parsed_statblocks, statblocks = results

    num_pages = len(statblocks.keys())
    logger.info("Found {} page{} of statblocks".format(num_pages, 's' if num_pages > 1 else ''))
    
    for page in parsed_statblocks:
        print("Page {}:".format(page))
        for creature in parsed_statblocks[page]:
            print()
            print(pretty_format_creature(creature))
            print()

..\data\pdfs\922210-OUTCLASSED_Expansion__Tashas_Panopticon_of_Nemeses.pdf
[2021-07-04 15:17:50,631] INFO     sbp              Loading creatures from ..\data\pdfs\922210-OUTCLASSED_Expansion__Tashas_Panopticon_of_Nemeses.pdf
[2021-07-04 15:17:52,499] INFO     sbp              Loaded file ..\data\pdfs\922210-OUTCLASSED_Expansion__Tashas_Panopticon_of_Nemeses.pdf
[2021-07-04 15:17:52,499] INFO     sbp              Loading ..\data\pdfs\922210-OUTCLASSED_Expansion__Tashas_Panopticon_of_Nemeses.pdf
[2021-07-04 15:17:52,528] INFO     sbp              Writing to file ../images.json
[2021-07-04 15:17:52,590] INFO     sbp              Found 7 pages of statblocks
Page 4:

Artifice Freelancer                                      freelancer does not require a free hand to benefit from
Medium humanoid, any alignment                           its shield.                                            
-------------------------------------------------------                                                

In [10]:
results[1][4][0].lines

[Line(id=323, text='Artifice Freelancer', bound=Bound(left=0.5390816319500291, top=0.1400743828933321, width=0.25902424497598175, height=0.025227270729166723), attributes=['statblock_title']),
 Line(id=324, text='Medium humanoid (any race), any alignment', bound=Bound(left=0.5390816319500291, top=0.15284544822537896, width=0.29905396225240777, height=0.012594696634311925), attributes=['race_type_header']),
 Line(id=325, text='Armor Class 20 (+1 black dragon scale mail, +1 shield', bound=Bound(left=0.5390816319500291, top=0.1850424165808078, width=0.3594384606439843, height=0.012594696634311927), attributes=['ac']),
 Line(id=326, text='of missile attraction)', bound=Bound(left=0.5553768150945925, top=0.19829999178598495, width=0.13951936264845774, height=0.012594696634311927), attributes=[]),
 Line(id=327, text='Hit Points 78 (12d8 + 24)', bound=Bound(left=0.5390816319500291, top=0.21155756699116154, width=0.1764015498371896, height=0.012594696634311927), attributes=['dice_roll', 'hp'])